In [11]:
# %load py_src/put_public_iris_to_local_minio.py
# %load py_src/put_public_iris_to_local_minio.py


# comment it to ease save/load
# %%writefile py_src/put_public_iris_to_local_minio.py
def put_public_iris_to_local_minio(args:dict) :

    import tempfile
    import os
    from minio import Minio
    import numpy as np
    import uuid
    import glob
    from sklearn import svm
    from sklearn import datasets
    from joblib import dump
    import numpy as np
    from sklearn.model_selection import train_test_split
    
    
    bucket_name = args.get("bucket_name", None)

    
    def upload_local_directory_to_minio(minio_client, local_path, bucket_name, minio_path):
        # assert os.path.isdir(local_path)

        for local_file in glob.glob(local_path + '/**'):
            local_file = local_file.replace(os.sep, "/") # Replace \ with / on Windows
            if not os.path.isfile(local_file):
                upload_local_directory_to_minio(
                    minio_client, local_file, bucket_name, minio_path + "/" + os.path.basename(local_file))
            else:
                remote_path = os.path.join(
                    minio_path, local_file[1 + len(local_path):])
                remote_path = remote_path.replace(
                    os.sep, "/")  # Replace \ with / on Windows
                minio_client.fput_object(bucket_name, remote_path, local_file)

    def get_minio_url():
        minio_host, minio_port = os.environ["MINIO_SERVICE_SERVICE_HOST"], os.environ["MINIO_SERVICE_SERVICE_PORT_HTTP"]
        minio_url= "{}:{}".format(minio_host, minio_port)
        return minio_url
    minio_url = get_minio_url()
    print("minio url:", minio_url)

    config = {"endpoint": minio_url,
        "access_key": "minio",
        "secret_key": "minio123",
        "secure": False}
    minio_client = Minio(**config)

    print("try to find bucket {}".format(bucket_name))
    found = minio_client.bucket_exists(bucket_name)
    print("found", found)
    if not found:
        minio_client.make_bucket(bucket_name)
    else:
        print("Bucket '{}' already exists".format(bucket_name))
    

    iris_data_dirpath="/tmp/"+str(uuid.uuid4())
    os.makedirs(iris_data_dirpath, exist_ok=True)
    iris = datasets.load_iris()
    X, y = iris.data, iris.target    
    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=123)
    

    
    np.savez("{}/iris.npz".format(iris_data_dirpath), x_train,y_train,x_test, y_test)
    
    
    upload_local_directory_to_minio(minio_client, iris_data_dirpath, bucket_name, "data/original")
                                   
    from shutil import rmtree
    rmtree(iris_data_dirpath)
    
